# Задание
Основные цели этого задания:

- Научиться генерировать негативы.

- Научиться настраивать алгоритмы коллаборативной фильтрации.

Задача:

Научиться рекомендовать пользователям фильмы на основе факта просмотра фильмов пользователями.

План решения:

1. Для решения задачи будем использовать те же данные, которые были использованы в скринкастах:

> a. [MovieLens](https://grouplens.org/datasets/movielens/) — источник данных.

> b. Предобработанные для обучения данные: история проставления оценок фильмам — ratings_df_sample_2.csv.

2. Предположим, постановка рейтинга — обязательное по итогам просмотра фильмов действие. Основываясь на этом, сгенерируйте новый целевой признак «факт просмотра фильма пользователем», который будет равен 1 для всех пар пользователь * фильм из подгруженного датасета.

3. А откуда взять «нолики»? В наших данных есть только пары пользователь * фильм, в которых пользователь точно смотрел фильм. Но для обучения модели нужны так называемые «негативы», то есть, пары, где пользователь фильм не смотрел. На практике приходится сталкиваться с необходимостью генерировать их вручную, давайте потренируемся это делать.

> a. Сначала найдите уникальные id всех пользователей и уникальные id всех фильмов.

> b. С помощью функции random.choice ([документация](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html)) сгенерируйте случайные пары пользователь * фильм

> c. Поскольку среди сгенерированных пар могут быть и такие, что пользователь в них уже смотрел фильм, сгенерируйте побольше пар, например, удвоенное количество строк из источника. Это может занять пару минут.

> d. Среди сгенерированных пар могут быть и дубликаты, удалите их.

> e. Оставьте среди сгенерированных пар только те, в которых пользователь фильм не смотрел.

> f. Возможно, пар получилось больше, чем нужно, выберите из них столько, сколько у нас строк в исходных данных.

> g. Добавьте очищенные сгенерированные пары к исходным данным. Значение целевого признака в них будет равно нулю. Убедитесь, что у вас не появились дубликаты в датасете.

4. Подготовьте датасет к обучению: отделите тестовую часть от тренировочной.

5. Обучите dummy-model. Пусть она будет возвращать случайную вероятность принадлежности классу 1. Для этого можете использовать функцию random.random ([документация](https://numpy.org/doc/stable/reference/random/generated/numpy.random.random.html)). Оцените ее качество какой-то метрикой на свой вкус. Необходимо прогнозировать именно вероятность, чтобы была возможность ранжировать по ней варианты для рекомендации лучшего контента пользователю.

6. Реализуйте три алгоритма коллаборативной фильтрации: user-, item-based и алгоритм на основе матричной факторизации. Оцените их качество и адекватность. Если качество недостаточно хорошее, попробуйте варьировать параметры: количество похожих пользователей/фильмов, количество элементов в матрицах при матричном разложении.

7. Опишите вывод, содержащий информацию о том, какой алгоритм проявил себя лучше всего.

In [1]:
# Импорт нужных библиотек
import pandas as pd
import numpy as np
import scipy.sparse as sp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from scipy.sparse.linalg import svds

pd.options.display.max_colwidth = 2000

<div class="alert alert-info">

1. Для решения задачи будем использовать те же данные, которые были использованы в скринкастах:

> a. MovieLens — источник данных.

> b. Предобработанные для обучения данные: история проставления оценок фильмам — ratings_df_sample_2.csv.

In [2]:
# загрузка данных
df = pd.read_csv('ratings_df_sample_2.csv')

In [3]:
df.head()

,userId,movieId,rating,timestamp
0,54,2,3.0,974918176
1,54,32,5.0,974836809
2,54,47,4.0,974837760
3,54,50,4.0,974837760
4,54,223,5.0,974840217


In [4]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040099 entries, 0 to 6040098
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   userId     6040099 non-null  int64  
 1   movieId    6040099 non-null  int64  
 2   rating     6040099 non-null  float64
 3   timestamp  6040099 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 184.3 MB


<div class="alert alert-success">
Никаких пропусков не обнаружено

<div class="alert alert-info">

2. Предположим, постановка рейтинга — обязательное по итогам просмотра фильмов действие. Основываясь на этом, сгенерируйте новый целевой признак «факт просмотра фильма пользователем», который будет равен 1 для всех пар пользователь * фильм из подгруженного датасета.

In [5]:
df['film_watch'] = 1

In [6]:
df.sample(10)

,userId,movieId,rating,timestamp,film_watch
75156,2992,3996,4.0,992929114,1
2705020,112988,5378,3.5,1116650055,1
5397984,56221,56174,2.5,1200915277,1
4003045,10292,180,5.0,1124502211,1
889547,36460,909,4.0,945584951,1
5081280,120416,1527,4.0,1020534269,1
4177230,28512,1249,3.0,944340285,1
570905,22966,2085,2.5,1083103994,1
4937105,105333,6373,2.0,1422372202,1
3321752,2311,593,5.0,955473595,1


Для удобства отмасштабируем идентификаторы фильмов и пользователей.

Этот метод позволит находить фильм по индексу матрицы (i-й столбец матрицы это i-й фильм)

In [7]:
%%time

movies_values = df['movieId'].unique()
df['movieId'] = df['movieId'].apply(lambda f: np.where(movies_values == f)[0][0])

CPU times: total: 36.3 s
Wall time: 42.6 s


Тоже самое сделаем и с ID пользователей

In [8]:
%%time

users_values = df['userId'].unique()
df['userId'] = df['userId'].apply(lambda f: np.where(users_values == f)[0][0])

CPU times: total: 1min 39s
Wall time: 1min 57s


In [9]:
df.head()

,userId,movieId,rating,timestamp,film_watch
0,0,0,3.0,974918176,1
1,0,1,5.0,974836809,1
2,0,2,4.0,974837760,1
3,0,3,4.0,974837760,1
4,0,4,5.0,974840217,1


<div class="alert alert-info">

3. А откуда взять «нолики»? В наших данных есть только пары пользователь * фильм, в которых пользователь точно смотрел фильм. Но для обучения модели нужны так называемые «негативы», то есть, пары, где пользователь фильм не смотрел. На практике приходится сталкиваться с необходимостью генерировать их вручную, давайте потренируемся это делать.

> a. Сначала найдите уникальные id всех пользователей и уникальные id всех фильмов.

> b. С помощью функции random.choice ([документация](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html)) сгенерируйте случайные пары пользователь * фильм

> c. Поскольку среди сгенерированных пар могут быть и такие, что пользователь в них уже смотрел фильм, сгенерируйте побольше пар, например, удвоенное количество строк из источника. Это может занять пару минут.

> d. Среди сгенерированных пар могут быть и дубликаты, удалите их.

> e. Оставьте среди сгенерированных пар только те, в которых пользователь фильм не смотрел.

> f. Возможно, пар получилось больше, чем нужно, выберите из них столько, сколько у нас строк в исходных данных.

> g. Добавьте очищенные сгенерированные пары к исходным данным. Значение целевого признака в них будет равно нулю. Убедитесь, что у вас не появились дубликаты в датасете.

In [10]:
# a. Сначала найдите уникальные id всех пользователей и уникальные id всех фильмов
unique_movies = df['movieId'].unique()
unique_users = df['userId'].unique()

In [11]:
%%time

# b. С помощью функции random.choice (документация) сгенерируйте случайные пары пользователь * фильм
generated_pairs = []
for _ in range(len(df) * 2):
    user = np.random.choice(unique_users)
    movie = np.random.choice(unique_movies)
    pair = (user, movie)
    generated_pairs.append(pair)

CPU times: total: 3min 1s
Wall time: 3min 39s


<div class="alert alert-warning">

c. Поскольку среди сгенерированных пар могут быть и такие, что пользователь в них уже смотрел фильм, сгенерируйте побольше пар, например, удвоенное количество строк из источника. Это может занять пару минут.

In [12]:
# d. Среди сгенерированных пар могут быть и дубликаты, удалите их.
generated_pairs = list(set(generated_pairs))

<div class="alert alert-warning">

f. Возможно, пар получилось больше, чем нужно, выберите из них столько, сколько у нас строк в исходных данных.

In [13]:
# Отбор нужного количества пар
filtered_pairs = generated_pairs[:len(df)]

In [14]:
# Создание нового DataFrame с сгенерированными парами
generated = pd.DataFrame(filtered_pairs, columns=['userId', 'movieId'])
generated['film_watch'] = 0

In [15]:
# Объединение сгенерированного DataFrame с исходными данными
merged_df = pd.concat([df, generated])

In [16]:
# Удаление дубликатов
merged_df = merged_df.drop_duplicates()

In [17]:
# Проверка наличия дубликатов
duplicates = merged_df.duplicated()
if duplicates.any():
    print("В датасете есть дубликаты")
else:
    print("Дубликатов в датасете нет")

Дубликатов в датасете нет


<div class="alert alert-info">

4. Подготовьте датасет к обучению: отделите тестовую часть от тренировочной.

In [18]:
merged_df

,userId,movieId,rating,timestamp,film_watch
0,0,0,3.0,974918176.0,1
1,0,1,5.0,974836809.0,1
2,0,2,4.0,974837760.0,1
3,0,3,4.0,974837760.0,1
4,0,4,5.0,974840217.0,1
...,...,...,...,...,...
6040094,13329,947,NaN,NaN,0
6040095,16787,821,NaN,NaN,0
6040096,16001,666,NaN,NaN,0
6040097,6138,395,NaN,NaN,0


Заполняем пропуски

In [19]:
merged_df.fillna(method='ffill', inplace=True)

In [20]:
merged_df

,userId,movieId,rating,timestamp,film_watch
0,0,0,3.0,9.749182e+08,1
1,0,1,5.0,9.748368e+08,1
2,0,2,4.0,9.748378e+08,1
3,0,3,4.0,9.748378e+08,1
4,0,4,5.0,9.748402e+08,1
...,...,...,...,...,...
6040094,13329,947,2.0,1.273010e+09,0
6040095,16787,821,2.0,1.273010e+09,0
6040096,16001,666,2.0,1.273010e+09,0
6040097,6138,395,2.0,1.273010e+09,0


**Мешаем данные для нормального распредения факта просмотра фильма**

In [21]:
merged_df = merged_df.sample(frac=1).reset_index(drop=True)

Снова отмасштабируем идентификаторы фильмов

In [22]:
merged_df

,userId,movieId,rating,timestamp,film_watch
0,17868,531,2.0,1.273010e+09,0
1,11642,443,3.0,9.486479e+08,1
2,4517,473,2.0,1.273010e+09,0
3,16422,162,2.0,1.273010e+09,0
4,14285,311,3.0,9.238551e+08,1
...,...,...,...,...,...
12080193,5524,550,2.0,1.273010e+09,0
12080194,6999,923,2.0,1.273010e+09,0
12080195,2468,967,2.0,1.273010e+09,0
12080196,15118,381,2.0,1.011299e+09,1


Пар вышло больше, чем нужно. Удалим лишние

In [23]:
merged_df = merged_df.sample(len(df))

In [24]:
merged_df

,userId,movieId,rating,timestamp,film_watch
8704021,17680,813,2.0,1.273010e+09,0
2704668,15869,895,2.0,1.273010e+09,0
8822668,1034,368,3.0,1.373130e+09,1
9398356,4262,720,2.5,1.170416e+09,1
3458235,4228,352,2.0,1.273010e+09,0
...,...,...,...,...,...
3445308,15473,316,2.0,1.273010e+09,0
10737191,3220,289,3.0,1.296196e+09,1
9739646,1363,360,5.0,9.703481e+08,1
6781554,9095,446,3.0,1.140542e+09,1


In [25]:
# определение переменных-предикторов и переменной ответа
X = merged_df.drop(['film_watch'], axis=1)
y = merged_df['film_watch']

In [26]:
y.value_counts()

0    3021318
1    3018781
Name: film_watch, dtype: int64

In [27]:
# Освобождаем память
del X
del y

**С рапределением вроде порядок**

In [28]:
# отделяем тестовую часть от тренировочной
train, test = train_test_split(merged_df, test_size=0.01, random_state=42)

In [29]:
X_train, X_test = train.drop(['film_watch'], axis=1), test.drop(['film_watch'], axis=1)
y_train, y_test = train['film_watch'], test['film_watch']

In [30]:
X_train.shape, X_test.shape

((5979698, 4), (60401, 4))

In [31]:
y_train.shape, y_test.shape

((5979698,), (60401,))

<div class="alert alert-info">

5. Обучите dummy-model. Пусть она будет возвращать случайную вероятность принадлежности классу 1. Для этого можете использовать функцию random.random ([документация](https://numpy.org/doc/stable/reference/random/generated/numpy.random.random.html)). Оцените ее качество какой-то метрикой на свой вкус. Необходимо прогнозировать именно вероятность, чтобы была возможность ранжировать по ней варианты для рекомендации лучшего контента пользователю.

In [32]:
# Создадим списки для хранения хранения метрик и их названий для итогового вывода
values = []
indexs = []

In [33]:
# Генерация случайных вероятностей для класса 1
def dummy_model(X):
    '''X - значения выбокри'''
    return tuple(np.random.random() for i in range(len(X)))

In [34]:
dummy_model_predict = dummy_model(X_test)

In [35]:
# Оценка качества модели по выбранной метрике(ROC AUC)
roc_auc = roc_auc_score(y_test, dummy_model_predict)
values.append(roc_auc)
indexs.append('dummy_model')
print("ROC AUC:", round(roc_auc, 4))

ROC AUC: 0.4956


<div class="alert alert-info">

6. Реализуйте три алгоритма коллаборативной фильтрации: user-, item-based и алгоритм на основе матричной факторизации. Оцените их качество и адекватность. Если качество недостаточно хорошее, попробуйте варьировать параметры: количество похожих пользователей/фильмов, количество элементов в матрицах при матричном разложении.

# Сформируем матрицу user-item

In [36]:
# Сформируем матрицу user-item
n_movies = train['movieId'].nunique()
n_movies

1000

In [37]:
n_users = train['userId'].nunique()
n_users

20000

In [38]:
%%time
train_data_matrix = np.array(pd.pivot_table(train, values='film_watch', index='userId', columns='movieId', fill_value=0))

CPU times: total: 7.53 s
Wall time: 11.2 s


In [39]:
train_data_matrix.shape

(20000, 1000)

In [40]:
train_data_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [41]:
%%time
# считаем попарное косинусное расстояние для пользователей (строк матрицы)
user_similarity = cosine_distances(train_data_matrix)

CPU times: total: 17.3 s
Wall time: 10.7 s


In [42]:
user_similarity

array([[0.        , 0.77026235, 0.72197797, ..., 0.89919762, 1.        ,
        0.96585154],
       [0.77026235, 0.        , 0.72919531, ..., 0.84541267, 0.90828398,
        0.93715722],
       [0.72197797, 0.72919531, 0.        , ..., 0.93453602, 0.88796295,
        0.95394009],
       ...,
       [0.89919762, 0.84541267, 0.93453602, ..., 0.        , 0.81122404,
        0.91684826],
       [1.        , 0.90828398, 0.88796295, ..., 0.81122404, 0.        ,
        0.87666603],
       [0.96585154, 0.93715722, 0.95394009, ..., 0.91684826, 0.87666603,
        0.        ]])

In [43]:
user_similarity.shape

(20000, 20000)

In [44]:
# # считаем попарное косинусное расстояние для фильмов (столбцов матрицы)
movie_similarity = cosine_distances(train_data_matrix.T)

In [45]:
movie_similarity

array([[0.        , 0.70063516, 0.69760829, ..., 0.88575543, 0.79217413,
        0.89509411],
       [0.70063516, 0.        , 0.61893819, ..., 0.8432421 , 0.77885869,
        0.84357833],
       [0.69760829, 0.61893819, 0.        , ..., 0.85196686, 0.76016194,
        0.86558694],
       ...,
       [0.88575543, 0.8432421 , 0.85196686, ..., 0.        , 0.88917489,
        0.83407833],
       [0.79217413, 0.77885869, 0.76016194, ..., 0.88917489, 0.        ,
        0.92827474],
       [0.89509411, 0.84357833, 0.86558694, ..., 0.83407833, 0.92827474,
        0.        ]])

In [46]:
movie_similarity.shape

(1000, 1000)

# Алгоритм user-based

**Для каждого пользователя найдем топ n ближайших соседей, исключающих самого себя**

In [47]:
def ton_n_nearest_neighbors(train_data_matrix, n_users=n_users, top=10):
    top=top
    top_similar_users = []
    for i in range(n_users):
        neighbors = (user_similarity[i]).argsort()[1:top + 1]
        top_similar_users.append(train_data_matrix[neighbors])
    top_similar_users = np.array(top_similar_users)
    return top_similar_users

**Попробуем сделать прогноз для... С потолка**
- 5 соседей
- 15 соседей
- 20 соседей

In [48]:
%%time

# 5 соседей
top_similar_users = ton_n_nearest_neighbors(train_data_matrix, n_users, 5)

CPU times: total: 22 s
Wall time: 33.7 s


In [49]:
top_similar_users.shape

(20000, 5, 1000)

In [50]:
predicted_watch_film_user_based = top_similar_users.mean(1)

In [51]:
predicted_watch_film_user_based.shape

(20000, 1000)

In [52]:
# Сделаем предсказание для датасетов
def round_to_nearest_0_5(n):
    return round(n * 2) / 2

In [53]:
test['film_watch_predict'] = test.apply(
    lambda f: round_to_nearest_0_5(predicted_watch_film_user_based[int(f['userId']), int(f['movieId'])]), axis = 1)

In [54]:
prediction = np.where(test['film_watch_predict'] >= 0.5 , 1, 0)
ROC_AUC_user_5 = roc_auc_score(prediction, test['film_watch'])
values.append(ROC_AUC_user_5)
indexs.append('Пользователей 5')

In [55]:
print(f"ROC AUC metric: {ROC_AUC_user_5}")

ROC AUC metric: 0.6042032775366779


In [56]:
# 15 соседей
top_similar_users = ton_n_nearest_neighbors(train_data_matrix, n_users, 15)
predicted_watch_film_user_based = top_similar_users.mean(1)
test['film_watch_predict'] = test.apply(
    lambda f: round_to_nearest_0_5(predicted_watch_film_user_based[int(f['userId']), int(f['movieId'])]), axis = 1)

prediction = np.where(test['film_watch_predict'] >= 0.5 , 1, 0)
ROC_AUC_user_15 = roc_auc_score(prediction, test['film_watch'])
values.append(ROC_AUC_user_15)
indexs.append('Пользователей 15')

print(f"ROC AUC metric: {ROC_AUC_user_15}")

ROC AUC metric: 0.637637931162996


In [57]:
# 20 соседей
top_similar_users = ton_n_nearest_neighbors(train_data_matrix, n_users, 20)
predicted_watch_film_user_based = top_similar_users.mean(1)
test['film_watch_predict'] = test.apply(
    lambda f: round_to_nearest_0_5(predicted_watch_film_user_based[int(f['userId']), int(f['movieId'])]), axis = 1)

prediction = np.where(test['film_watch_predict'] >= 0.5 , 1, 0)
ROC_AUC_user_20 = roc_auc_score(prediction, test['film_watch'])
values.append(ROC_AUC_user_20)
indexs.append('Пользователей 20')

print(f"ROC AUC metric: {ROC_AUC_user_20}")

ROC AUC metric: 0.6410958851278092


<div class="alert alert-success">
Пока самый лучший результат показал алгоритм user-based с 5 ближашими кино-пользователями

# Алгоритм item-based

In [58]:
# 5 соседних фильмов
top = 5 
top_similar_films = []
for i in range(n_movies):
    neighbors = (movie_similarity[i]).argsort()[1:top + 1]
    top_similar_films.append(train_data_matrix.T[neighbors])
top_similar_films = np.array(top_similar_films)

In [59]:
top_similar_films.shape

(1000, 5, 20000)

In [60]:
predicted_films_item_based = top_similar_films.mean(1).T
predicted_films_item_based.shape

(20000, 1000)

In [61]:
test['film_watch_item_predict'] = test.apply(
    lambda f: round_to_nearest_0_5(predicted_films_item_based[int(f['userId']), int(f['movieId'])]), axis = 1)

In [62]:
prediction = np.where(test['film_watch_item_predict'] >= 0.5 , 1, 0)
ROC_AUC_user_5_item = roc_auc_score(prediction, test['film_watch'])
values.append(ROC_AUC_user_5_item)
indexs.append('Фильмов 5')
print(f"ROC AUC metric: {ROC_AUC_user_5_item}")

ROC AUC metric: 0.6282373514582213


In [63]:
# 15 соседних фильмов
top = 15 
top_similar_films = []
for i in range(n_movies):
    neighbors = (movie_similarity[i]).argsort()[1:top + 1]
    top_similar_films.append(train_data_matrix.T[neighbors])
top_similar_films = np.array(top_similar_films)

predicted_films_item_based = top_similar_films.mean(1).T

test['film_watch_item_predict'] = test.apply(
    lambda f: round_to_nearest_0_5(predicted_films_item_based[int(f['userId']), int(f['movieId'])]), axis = 1)

prediction = np.where(test['film_watch_item_predict'] >= 0.5 , 1, 0)
ROC_AUC_user_15_item = roc_auc_score(prediction, test['film_watch'])
values.append(ROC_AUC_user_15_item)
indexs.append('Фильмов 15')
print(f"ROC AUC metric: {ROC_AUC_user_15_item}")

ROC AUC metric: 0.6561933846818835


In [64]:
# 20 соседних фильмов
top = 20 
top_similar_films = []
for i in range(n_movies):
    neighbors = (movie_similarity[i]).argsort()[1:top + 1]
    top_similar_films.append(train_data_matrix.T[neighbors])
top_similar_films = np.array(top_similar_films)

predicted_films_item_based = top_similar_films.mean(1).T

test['film_watch_item_predict'] = test.apply(
    lambda f: round_to_nearest_0_5(predicted_films_item_based[int(f['userId']), int(f['movieId'])]), axis = 1)

prediction = np.where(test['film_watch_item_predict'] >= 0.5 , 1, 0)
ROC_AUC_user_20_item = roc_auc_score(prediction, test['film_watch'])
values.append(ROC_AUC_user_20_item)
indexs.append('Фильмов 20')
print(f"ROC AUC metric: {ROC_AUC_user_20_item}")

ROC AUC metric: 0.6536594927853112


<div class="alert alert-success">
Все равно пока самым лучшим результатом является алгоритм user-based с 5 ближашими кино-пользователями

# Алгоритм на основе матричного разложения. Сделаем SVD.

In [65]:
# Алгоритм на основе матричного разложения (для 5, 15 и 20 значений)
train_data_matrix.shape

(20000, 1000)

In [66]:
%%time
# делаем SVD (5 значений)
u, s, vh = svds(train_data_matrix, k=5)
s_diag_matrix = np.diag(s)

users = np.dot(u, s_diag_matrix)
items = vh.T

CPU times: total: 4.39 s
Wall time: 1.4 s


In [67]:
test['svd_predictions'] = test.apply(
    lambda f: round_to_nearest_0_5(np.dot(users[int(f['userId'])], items[int(f['movieId'])])), axis = 1)

In [68]:
prediction = np.where(test['svd_predictions'] >= 0.5 , 1, 0)
ROC_AUC_SVD_5 = roc_auc_score(prediction, test['film_watch'])
values.append(ROC_AUC_SVD_5)
indexs.append('Разложение 5')
print(f"ROC AUC metric: {ROC_AUC_SVD_5}")

ROC AUC metric: 0.6515141275910707


In [69]:
# делаем SVD (15 значений)
u, s, vh = svds(train_data_matrix, k=15)
s_diag_matrix = np.diag(s)

users = np.dot(u, s_diag_matrix)
items = vh.T
test['svd_predictions'] = test.apply(
    lambda f: round_to_nearest_0_5(np.dot(users[int(f['userId'])], items[int(f['movieId'])])), axis = 1)

prediction = np.where(test['svd_predictions'] >= 0.5 , 1, 0)
ROC_AUC_SVD_15 = roc_auc_score(prediction, test['film_watch'])
values.append(ROC_AUC_SVD_15)
indexs.append('Разложение 15')
print(f"ROC AUC metric: {ROC_AUC_SVD_15}")

ROC AUC metric: 0.6673719960785238


In [70]:
# делаем SVD (15 значений)
u, s, vh = svds(train_data_matrix, k=20)
s_diag_matrix = np.diag(s)

users = np.dot(u, s_diag_matrix)
items = vh.T
test['svd_predictions'] = test.apply(
    lambda f: round_to_nearest_0_5(np.dot(users[int(f['userId'])], items[int(f['movieId'])])), axis = 1)

prediction = np.where(test['svd_predictions'] >= 0.5 , 1, 0)
ROC_AUC_SVD_20 = roc_auc_score(prediction, test['film_watch'])
values.append(ROC_AUC_SVD_20)
indexs.append('Разложение 20')
print(f"ROC AUC metric: {ROC_AUC_SVD_20}")

ROC AUC metric: 0.66549932528804


<div class="alert alert-info">

7. Опишите вывод, содержащий информацию о том, какой алгоритм проявил себя лучше всего.

In [71]:
data = {'Показатели метрик ROC AUC' : values}
data = pd.DataFrame(data, index=indexs)

In [72]:
data.round(4)

,Показатели метрик ROC AUC
dummy_model,0.4956
Пользователей 5,0.6042
Пользователей 15,0.6376
Пользователей 20,0.6411
Фильмов 5,0.6282
Фильмов 15,0.6562
Фильмов 20,0.6537
Разложение 5,0.6515
Разложение 15,0.6674
Разложение 20,0.6655


<div class="alert alert-success">
    
**Вывод**
    
Итак, были реализованы три алгоритма коллаборативной фильтрации: user-, item-based и алгоритм на основе матричной факторизации 
с разными количеством значений: 5, 15, 20.
По итогам лучше себя лусше себя показал алгоритм на основе матричной факторизации(SVD) со значением 15, значит он лучше справляеься с задачей
рекомандации фильмов пользователю.